# head

In [1]:
%cd ~/Workspace/kaggle-2019Q3-cellular/
!nvidia-smi

/home/chengjiun/Workspace/kaggle-2019Q3-cellular
Tue Jul  9 12:18:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   53C    P8    11W / 250W |    349MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                           

In [2]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import torchvision
from torchvision import transforms as T

import tqdm

import warnings
warnings.filterwarnings('ignore')

# setup

In [7]:
from preprocess import ImagesDS
from utils import accuracy
from Models.basic import DensNet
from postprocess import prediction
from train import Train

class MyResNet(nn.Module):
    def __init__(self, num_classes=1000, num_channels=6):
        super().__init__()
        model = torchvision.models.resnet50(pretrained=True)
        model.fc = nn.Linear(2048, num_classes, bias=True)
        # freeze parameters     
        self.model = model

    def freeze_parameters(self):
        for param in self.model.parameters():
            param.requres_grad = False
        
        for param in self.model.layer4.parameters():
            param.requires_grad = True
        for param in self.model.avgpool.parameters():
            param.requires_grad = True
        for param in self.model.fc.parameters():
            param.requires_grad = True
    
        
    def forward(self, x):
        return self.model(x)



In [8]:
path_data = '../DATA/kaggle-2019Q3-cellular/'
batch_size = 8
device = 'cuda'
classes = 1108

In [9]:

#model = MyResNet(num_classes=classes)
#model.freeze_parameters()
model = DensNet(num_classes=classes)
model.to(device)

DensNet(
  (features): Sequential(
    (conv0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (c

In [5]:
ds = ImagesDS(path_data+'/train.csv', path_data)
ds_test = ImagesDS(path_data+'/test.csv', path_data, mode='test')

# train model

In [7]:
TrainEngine = Train(ds, DensNet, batch_size, num_workers, device)

Epoch 1 -> Train Loss: 0.0048, ACC: 22.23%
Epoch 2 -> Train Loss: 0.0045, ACC: 26.54%
Epoch 3 -> Train Loss: 0.0043, ACC: 29.27%
Epoch 4 -> Train Loss: 0.0042, ACC: 31.92%
Epoch 5 -> Train Loss: 0.0040, ACC: 34.70%
Epoch 6 -> Train Loss: 0.0039, ACC: 37.37%
Epoch 7 -> Train Loss: 0.0037, ACC: 40.91%
Epoch 8 -> Train Loss: 0.0036, ACC: 43.05%
Epoch 9 -> Train Loss: 0.0034, ACC: 45.62%
Epoch 10 -> Train Loss: 0.0033, ACC: 48.50%
Epoch 11 -> Train Loss: 0.0031, ACC: 51.17%
Epoch 12 -> Train Loss: 0.0030, ACC: 53.69%
Epoch 13 -> Train Loss: 0.0028, ACC: 56.27%
Epoch 14 -> Train Loss: 0.0027, ACC: 58.96%
Epoch 15 -> Train Loss: 0.0026, ACC: 61.64%
Epoch 16 -> Train Loss: 0.0025, ACC: 63.18%
Epoch 17 -> Train Loss: 0.0023, ACC: 66.13%
Epoch 18 -> Train Loss: 0.0022, ACC: 68.17%
Epoch 19 -> Train Loss: 0.0021, ACC: 70.14%
Epoch 20 -> Train Loss: 0.0020, ACC: 72.87%
Epoch 21 -> Train Loss: 0.0019, ACC: 74.59%
Epoch 22 -> Train Loss: 0.0018, ACC: 76.26%
Epoch 23 -> Train Loss: 0.0017, ACC: 78.5

# prediction test

In [9]:
tloader = D.DataLoader(ds_test, batch_size=batch_size,
                       shuffle=False, num_workers=2)

preds = prediction(model, tloader)

In [10]:
submission = pd.read_csv(path_data + '/test.csv')
submission['sirna'] = preds.astype(int)
submission.to_csv('submission.2.densenet_deep.csv', index=False, columns=['id_code','sirna'])